In [46]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [7, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
from sklearn import metrics
# from scipy.special import rel_entr
from scipy.spatial.distance import jensenshannon, euclidean

In [82]:
def dist(A, B, method = "js"):
    aCols = list(A.columns)
    bCols = list(B.columns)
    aRows = list(A.index)
    bRows = list(B.index)
    intersectCols = list(set(aCols) & set(bCols))
    intersectRows = list(set(aRows) & set(bRows))

    A = A.loc[intersectRows, intersectCols]
    B = B.loc[intersectRows, intersectCols]
    A = A.fillna(0)
    B = B.fillna(0)
    
    if method == 'ed':
        distList = [ed(A[sample].to_numpy().astype(np.float32), B[sample].to_numpy().astype(np.float32))
                    for sample in intersectCols]
    elif method == 'js':
        distList = [js(A[sample].to_numpy().astype(np.float32), B[sample].to_numpy().astype(np.float32))
                    for sample in intersectCols]
    else:
        distList = [ks(A[sample].to_numpy().astype(np.float32), B[sample].to_numpy().astype(np.float32))
                    for sample in intersectCols]
    dists = pd.Series(distList)
    dists.index = intersectCols
    dist = np.sqrt(np.sum(dists.values ** 2))
    return(dist)

In [83]:
def ed(a, b):
    if len(a) != len(b):
        print("The two arrays are not same size!")
    if np.sum(a) == 0 or np.sum(b) == 0:
        return(np.nan)
    a /= np.sum(a)
    b /= np.sum(b)
    dist = euclidean(a, b)
    return(dist)


def js(a, b):
    if len(a) != len(b):
        print("The two arrays are not same size!")
    if np.sum(a) == 0 or np.sum(b) == 0:
        return(np.nan)
    a /= np.sum(a)
    b /= np.sum(b)
    dist = jensenshannon(a, b)
    return(dist)


def ks(a, b):
    if len(a) != len(b):
        print("The two arrays are not same size!")
    if np.sum(a) == 0 or np.sum(b) == 0:
        return(np.nan)     
    a /= np.sum(a)
    b /= np.sum(b)
    dist = np.amax(np.abs(np.cumsum(a) - np.cumsum(b)))
    return(dist)

In [85]:
algs = ["cibersort", "mcpcounter", "epic", "xcell"]
sigs = ["LM22", "LM7c", "LM9", "PBMC"]
distTypes = ["ed", "js", "ks"]
cancerTypes = ["brca", "ccrcc", "colon", "endometrial", "hnscc", "luad", "ovarian"]
for distType in distTypes:
    for sig in sigs:
        for cancerType in cancerTypes:
            matrix = []
            for algA in algs:
                tempM = []
                for algB in algs:
                    mA = pd.read_csv("../fig4/data/" + cancerType + "-all-mrna-raw-" + algA + "-" + sig + ".tsv", sep='\t', index_col=0)
                    mB = pd.read_csv("../fig4/data/" + cancerType + "-all-prot-raw-" + algB + "-" + sig + ".tsv", sep='\t', index_col=0)
                    tempM.append(dist(mA, mB, distType))
                matrix.append(tempM)
            matrix = np.array(matrix)
            ax = plt.figure()
            ax = sns.heatmap(matrix, cmap='coolwarm', annot = True, xticklabels = algs, yticklabels = algs)
            ax.set_title(cancerType + '-' + sig + '-' + distType)
            ax.get_figure().savefig(distType + '-' + sig + '-' + cancerType + ".pdf", bbox_inches='tight')
            plt.close()